1. Run MNIST
- Add a text cell and comment on network training and test accuracy
- Train for 20 epochs and evaluate. Comment on your findings
- The first layer transforms the 784-element image vector to a 512 dimensional intermediate representation Experiment with different intermediate dimensions. Make a markdown table of network performance on the test set for varying intermediate dimension. Comment on your results
- Replace network compilation with 
```
from tensorflow.keras import optimizers
network.compile(optimizer=optimizers.RMSprop(lr=0.001, momentum=0.0),
                loss='categorical_crossentropy', 
                metrics=['accuracy'])
```
The code is exactly equivalent, but we are now able to adjust learning rate and momentum. `lr=0.001` is the default value: experiment with different learning rates. Tabulate your results and interpret
- Experiment with different momentums. Tabulate and interpret

In [1]:
# MNIST

# load
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# preprocess
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

from tensorflow.keras.utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# build
from tensorflow.keras import models, layers
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28, )))
network.add(layers.Dense(10, activation='softmax'))


from tensorflow.keras import optimizers
network.compile(optimizer=optimizers.RMSprop(lr=0.001, momentum=0.0),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
# train
network.fit(train_images, train_labels, epochs=20, batch_size=128)

11493376/11490434 [==============================] - 0s 0us/step
Epoch 1/20
469/469 [==============================] - 35s 75ms/step - loss: 0.2580 - accuracy: 0.9251
Epoch 2/20
469/469 [==============================] - 40s 85ms/step - loss: 0.1034 - accuracy: 0.9699
Epoch 3/20
469/469 [==============================] - 36s 78ms/step - loss: 0.0682 - accuracy: 0.9791
Epoch 4/20
469/469 [==============================] - 34s 73ms/step - loss: 0.0502 - accuracy: 0.9847
Epoch 5/20
469/469 [==============================] - 43s 91ms/step - loss: 0.0378 - accuracy: 0.9882
Epoch 6/20
469/469 [==============================] - 36s 76ms/step - loss: 0.0290 - accuracy: 0.9910
Epoch 7/20
469/469 [==============================] - 33s 71ms/step - loss: 0.0219 - accuracy: 0.9935
Epoch 8/20
469/469 [==============================] - 40s 86ms/step - loss: 0.0172 - accuracy: 0.9950
Epoch 9/20
469/469 [==============================] - 38s 81ms/step - loss: 0.0132 - accuracy: 0.9962
Epoch 10/20
469/4

In [2]:
# evaluate on the test set
test_loss, test_acc = network.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 4ms/step - loss: 0.0952 - accuracy: 0.9817


In [5]:
import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

NameError: name 'history' is not defined